<h1 style="color: rgb(241, 90, 36)"><img src="./images/SQLIcon.png?modified=24223" width=80px height=80px style="vertical-align: middle;"> Window Functions</h1>

> Whereas aggregate functions operate on entire datasets, flattening tables and collapsing multiple rows into a single result, *window functions* operate within a defined 'window' of rows, adding a new column entry to each row. This functionality is facilitated through three fundamental mechanisms: *partitioning*, *ordering*, and *framing*.

- **Partitioning** 
    - Divides the dataset into distinct groups or partitions based on specific criteria
    - Within each partition, window functions will operate independently, performing calculations on the rows within that group
    - For example, in a sales dataset, partitioning by product category would enable separate calculations of metrics such as sales performance for each category

- **Ordering**
    - Establishes a sequence for the rows within the partition, providing context and direction to the calculations performed by window functions
    - By defining a specific order, analysts can uncover data patterns and trends
    - Continuing with the sales examples, ordering by date would enable the calculation of cumulative sales over time, revealing insights into sales trends and seasonality

- **Framing**
    - Determines which rows are included in the calculation at any given point. It defines the boundaries of the window of rows within which window functions operate.
    - The mechanism allows for dynamic analysis, where the window can shift or remain fixed based on specified criteria
    - For instance, framing can be used to compute moving averages over a rolling window of the past N days, providing insights into short-term trends


Together, these three mechanisms enable SQL users to conduct complex analyses that go beyond simple aggregations.

<h2 style="color: rgb(241, 90, 36)"> Basic Syntax</h2>

The basic syntax of a window function without any additional clauses, looks like this:

```sql
SELECT 
    column1,
    column2,
    window_function(column3) OVER () AS result_column
FROM
    table_name;
```

In the syntax above, the `window_function` represents the specific window function you want to apply, and `column3` is the column you are performing the calculation on. The `OVER()` clause indicates that the window function operates over the entire result without any partitioning or ordering.

Let's look at an example window function without partitioning or ordering:

```sql
-- Example without partitioning or ordering
SELECT 
    product_id,
    sales_date,
    sales_amount,
    SUM(sales_amount) OVER () AS total_sales
FROM
    sales_data;
```
The output of this query would look like this:

<table border="1">
  <tr>
    <th>product_id</th>
    <th>sales_date</th>
    <th>sales_amount</th>
    <th>total_sales</th>
  </tr>
  <tr>
    <td>1</td>
    <td>2024-01-01</td>
    <td>1000</td>
    <td>5000</td>
  </tr>
  <tr>
    <td>2</td>
    <td>2024-01-02</td>
    <td>1500</td>
    <td>5000</td>
  </tr>
  <tr>
    <td>3</td>
    <td>2024-01-03</td>
    <td>2000</td>
    <td>5000</td>
  </tr>
  <tr>
    <td>1</td>
    <td>2024-01-04</td>
    <td>500</td>
    <td>5000</td>
  </tr>
  <tr>
    <td>2</td>
    <td>2024-01-05</td>
    <td>1000</td>
    <td>5000</td>
  </tr>
</table>


This query calculates the total sales amount across all records in the `sales_data` table. However, this is essentially equivalent to a simple aggregation using a traditional aggregate function like `SUM()`. In other words, while no partitioning or ordering is specified in the `OVER` clause, the window function operates on the entire result set of a single partition, considering all rows together. Consequently, the `total_sales` column will contain the same value for every row in the result set, as it represents the cumulative sum of `sales_amount` across the entire dataset.

In the next sections we will introduce partitioning and ordering.

<h3 style="color: rgb(241, 90, 36)">Partitioning Syntax</h3>

To introduce partitioning to a window function, you need to use the `PARTITION BY` clause:

```sql
SELECT 
    column1,
    column2,
    window_function(column3) OVER (PARTITION BY partition_column) AS result_column
FROM
    table_name;
```

Here, `partition_column` is the column by which you want to partition the data. This divides the dataset into separate partitions, and the window function is applied independently within each partition.

Let's look at an example window function with partitioning:

```sql
-- Example with partitioning by product category
SELECT 
    product_id,
    sales_date,
    sales_amount,
    SUM(sales_amount) OVER (PARTITION BY product_category) AS category_sales
FROM
    sales_data;
```

The output of this query would look like this:

<table border="1">
  <tr>
    <th>product_id</th>
    <th>sales_date</th>
    <th>sales_amount</th>
    <th>category_sales</th>
  </tr>
  <tr>
    <td>1</td>
    <td>2024-01-01</td>
    <td>1000</td>
    <td>3500</td>
  </tr>
  <tr>
    <td>2</td>
    <td>2024-01-02</td>
    <td>1500</td>
    <td>2500</td>
  </tr>
  <tr>
    <td>3</td>
    <td>2024-01-03</td>
    <td>2000</td>
    <td>4000</td>
  </tr>
  <tr>
    <td>1</td>
    <td>2024-01-04</td>
    <td>500</td>
    <td>3500</td>
  </tr>
  <tr>
    <td>2</td>
    <td>2024-01-05</td>
    <td>1000</td>
    <td>2500</td>
  </tr>
</table>


In this example, the `OVER(PARTITION BY product_category)` clause partitions the data by `product_category`, creating separate groups for each distinct category. The `SUM(sales_amount)` window function calculates the total sales amount within each partition, which results in the `category_sales` column. 

As a consequence of partitioning, the same aggregation value will be repeated for entries with the same `product_category`. For instance, if there are multiple rows with `product_category` equal to `Electronics`, the `category_sales` value will be the same for all of them, representing the total sales amount within the `Electronics` category.

In the next section, we will see how to use the `ORDER BY` clause within the `OVER` clause to change this behaviour.

<h3 style="color: rgb(241, 90, 36)"> Ordering Syntax</h3>

To add ordering to a window function, you need to use the `ORDER BY` clause:

```sql
SELECT 
    column1,
    column2,
    window_function(column3) OVER (PARTITION BY partition_column ORDER BY order_column) AS result_column
FROM
    table_name;
```

In this example, `order_column` specifies the column by which you want to order the rows within each partition. This ensures that the window function operates on the rows in a specific sequence within each partition.

Let's look at an example window function with ordering:

```sql
-- Example with ordering by sales date within each product category
SELECT 
    product_id,
    sales_date,
    sales_amount,
    SUM(sales_amount) OVER (PARTITION BY product_category ORDER BY sales_date) AS cumulative_sales
FROM
    sales_data;
```

The output of this query would look like:

<table border="1">
  <tr>
    <th>product_id</th>
    <th>sales_date</th>
    <th>sales_amount</th>
    <th>cumulative_sales</th>
  </tr>
  <tr>
    <td>1</td>
    <td>2024-01-01</td>
    <td>1000</td>
    <td>1000</td>
  </tr>
  <tr>
    <td>2</td>
    <td>2024-01-02</td>
    <td>1500</td>
    <td>1500</td>
  </tr>
  <tr>
    <td>3</td>
    <td>2024-01-03</td>
    <td>2000</td>
    <td>2000</td>
  </tr>
  <tr>
    <td>1</td>
    <td>2024-01-04</td>
    <td>500</td>
    <td>1500</td>
  </tr>
  <tr>
    <td>2</td>
    <td>2024-01-05</td>
    <td>1000</td>
    <td>2500</td>
  </tr>
</table>


In this example, the `OVER (PARTITION BY product_category ORDER BY sales_date)` clause partitions data by `product_category` and orders the rows within each partition by `sales_date`. The `SUM(sales_amount)` window function then calculates the cumulative sales amount within each partition, resulting in the `cumulative_sales` column. By ordering the rows by `sales_date` within each product category, the query computes the cumulative sales over time for each category.

<h3 style="color: rgb(241, 90, 36)"> Framing Syntax</h3>

Framing in window functions allows you to define a subset of rows within the window over which the window function operates. This subset is specified using the `ROWS BETWEEN` clause, which determines the range of rows relative to the current row.

The syntax for the `ROWS BETWEEN` clause is as follows:

```sql
ROWS BETWEEN <start> AND <end>
```

Where `<start>` and `<end>` define the boundaries of the frame relative to the current row. These boundaries can be specified using various options:
- `UNBOUNDED PRECEDING`: Represents the first row of the partition
- `CURRENT ROW`: Represents the current row
- `UNBOUNDED FOLLOWING`: Represents the last row of the partition
- A numerical value: Represents a specific number of rows before (`PRECEDING`) or after (`FOLLOWING`) the current row

Additionally, you can use `PRECEDING` and `FOLLOWING` keywords to specify rows before or after the current row, respectively:

```sql
SELECT 
    column1,
    column2,
    window_function(column3) OVER (PARTITION BY partition_column ORDER BY order_column ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS result_column
FROM
    table_name;
```

Here, the `ROWS BETWEEN 3 PRECEDING AND CURRENT ROW` specifies the frame of rows within each partition to include in the calculation. This defines a moving window of the current row plus the three preceding rows.

Let's look at an example window function that uses framing:

```sql
-- Example with frame specification for calculating moving average
SELECT 
    product_id,
    sales_date,
    sales_amount,
    AVG(sales_amount) OVER (PARTITION BY product_category ORDER BY sales_date ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS moving_avg
FROM
    sales_data;
```

In this example, the `ROWS BETWEEN 3 PRECEDING AND CURRENT ROW` clause within the `OVER` clause specifies the frame of rows for the window function. The frame includes the current row and the three preceding rows, ordered by `sales_data`, within each `product_category` partition. The `AVG(sales_amount)` window function then computers the moving average of sales amount within this frame, resulting in the `moving_avg` columns.

<h2 style="color: rgb(241, 90, 36)"> Common Window Functions</h2>

In this section we will look at some of the most common window functions.

### `SUM()`, `AVG()`, `MAX()`, `MIN()`

These are traditional aggregate functions in SQL, but when used as window functions, they operate on a set of rows defined by the window rather than the entire dataset. They enable the calculation of running totals, averages, maximums, minimums over a specific subset of data within partitions.

### `LEAD()` and `LAG()`

The `LEAD()` and `LAG()` functions allow access to data from subsequent or preceding rows within a partition, respectively. They are commonly used to compare current row values with those of adjacent rows, or to calculate difference over time.

`LEAD()` allows you to retrieve the value of a column from the next row, while `LAG()` retrieves the value from the previous row.

Let's consider a sales dataset where you want to calculate the difference in sales amount between consecutive days:

```sql
SELECT
    sales_date,
    sales_amount,
    LEAD(sales_amount) OVER (ORDER BY sales_date) - sales_amount AS sales_difference
FROM
    sales_data;
```
In this example, `LEAD(sales_amount)` retrieves the sales amount from the next row, allowing you to calculate the difference, `sales_difference`, between consecutive days. Similarly, you can use `LAG()` to compare the current's row sales amount with the previous day's sales.

### `ROW_NUMBER()`

The `ROW_NUMBER()` function is a fundamental window function that assigns a unique sequential integer to each row within a partition, based on the specified ordering criteria. It provides a simple way to enumerate rows, which is particularly useful for ranking and pagination tasks. Each row receives a distinct number starting from 1 for the first row in the partition, incrementing by 1 for each subsequent row.

Let's consider a scenario where you want to assign a unique identifier to each customer order in a sales dataset:

```sql
SELECT
    order_id,
    customer_id,
    order_date,
    ROW_NUMBER() OVER (ORDER BY order_date) AS order_sequence_number
FROM
    orders;
```
In this example, `ROW_NUMBER() OVER(ORDER BY order_date)` generates a sequential integer (`order_sequence_number`) for each row in the result set, based on the ordering of `order_date`. Each order receives a distinct number, starting from 1 for the earliest order date, and incrementing by 1 for each subsequent order. This provides a simple way to enumerate orders based on their chronological order, facilitating tasks such as ranking or identifying the first order for each customer.

<h2 style="color: rgb(241, 90, 36)"> Common Use Cases</h2>

Throughout this section, we will use the Pagila database to demonstrate practical examples of window functions.

<h3 style="color: rgb(241, 90, 36)"> Running Totals</h3>

Running totals are commonly used to track cumulative values over time, providing insights into trends and patterns in data.

Let's look at how to calculate the running total of sales amounts for each customer in the Pagila database:

```sql
SELECT
    customer_id,
    payment_date,
    amount,
    SUM(amount) OVER (PARTITION BY customer_id ORDER BY payment_date) AS running_total
FROM
    payment;
```
<p align=center> <img src=images/RunningTotals.png width=600> </p>


This query computes the running total of payment amounts for each customer, ordered by payment date. The `SUM(amount) OVER (PARTITION BY customer_id ORDER BY payment_date)` window functions calculate the cumulative sum of payments amounts for each customer, providing insights into their spending behaviour over time.

<h3 style="color: rgb(241, 90, 36)"> Moving Averages</h3> 

Moving averages are useful for smoothing out fluctuations in data and identifying trends over time. With window functions, you can easily calculate moving averages over a specified window, allowing for more insightful analysis of time-series data.

Let's look at an example of computing the 7-day moving average of payment amounts over a 7-day window (3 days before and 3 days after each payment date). This can help us understand payment trends over time:

```sql
SELECT
    payment_date,
    amount,
    AVG(amount) OVER (ORDER BY payment_date ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS moving_average
FROM
    payment;

```
<p align=center> <img src=images/MovingAverages.png width=600> </p>


This query calculates the moving average of payment amounts over a 7-day window. The `AVG(amount) OVER (ORDER BY payment_date ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING)` window function computes the average payment amount for each row and the three preceding and following rows. This approach helps to smooth out short-term fluctuations in payment amounts, providing a clearer view of the overall trend.

<h3 style="color: rgb(241, 90, 36)"> Top N per Groups</h3>  

Identifying the top N records within each group is useful in scenarios like identifying the best-performing products in each category or the top customers by sales. Let's look at an example of retrieving the top 3 films by rental count in each category:

```sql
SELECT
    category_id,
    film_id,
    title,
    rental_count,
    ROW_NUMBER() OVER (PARTITION BY category_id ORDER BY rental_count DESC) AS rank
FROM (
    SELECT
        f.category_id,
        f.film_id,
        f.title,
        COUNT(r.rental_id) AS rental_count
    FROM
        film AS f
    JOIN
        inventory AS i ON f.film_id = i.film_id
    JOIN
        rental AS r ON i.inventory_id = r.inventory_id
    GROUP BY
        f.category_id,
        f.film_id,
        f.title
) AS rentals_by_category
WHERE
    rank <= 3;
```
<p align=center> <img src=images/TopN.png width=600> </p>

In the example above, the inner subquery calculates the total number of rentals for each film and groups them by category. This subquery produces a result set with columns `category_id`, `film_id`, `title`, and `rental_count`, representing the number of times each film has been rented. The outer query uses the results from the inner subquery to rank films within each category based on their rental counts and finally filters for the top 3 films in each category. More specifically:
- `ROW_NUMBER() OVER (PARTITION BY c.category_id ORDER BY COUNT(r.rental_id) DESC) AS rank`: This window function assigns a unique sequential integer to each row within a category partition, ordered by the rental count in descending order. Each film within a category receives a rank based on its rental count, with the highest rental count receiving rank 1.
- `WHERE rank <= 3`: Filters the results to include only the top 3 films within each category based on their rental counts. This ensures that only the most popular films in each category are selected.

## Key Takeaways

- Window functions enable focused analysis within a specified range of rows in our dataset
- Partitioning segments data into logical groups enabling separate computations within each group, which is particularly useful for comparative analyses across distinct categories or segments
- Ordering arranges rows within partitions, ensuring that calculations and comparisons proceed in a structured manner, crucial for tasks like ranking or identifying trends
- Framing specifies the window of rows for calculations, allowing for dynamic analysis over sliding windows of data, facilitating operations such as moving averages or identifying outliers
- Aggregate functions like `SUM`, `AVG`, `MAX`, and `MIN` can operate as window functions, enabling calculations like running totals and averages
- `ROW_NUMBER()` assigns unique identifiers to rows within partitions, facilitating tasks such as pagination or ranking, ensuring consistent ordering of results
- `LEAD()` and `LAG()` retrieve data from subsequent or preceding rows within partitions, providing contextual information for trend analysis or comparative assessments